As part of the work to make it easier for dataset depositors to specify how they're data should be access and used by letting them choose from a pre-defined list of licenses, questions have been brought up about the relationship between waivers/licenses and the other Terms metadata fields (e.g. Terms of Use, Terms of Access, Conditions) that the Dataverse software ships with.

This notebook explores the Terms metadata of Dataverse repositories to help explore that relationship in practice. It identifies published datasets with both:
- a CC0 waiver or a license and
- metadata entered in one or more of the other Terms metadata fields

to help answer the following questions:

- When depositors publish datasets with a CC0 waiver or a license, how often are they filling the other Terms fields?
- When depositors publish datasets with a CC0 waiver or a license and fill one or more other Terms fields, are they entering metadata that conflicts or might conflict with the license?

## Considerations
- This is an exploration of depositor behavior and not necessarily what they "should" be doing.
- The data used in this exploration comes from 49 of 67 known Dataverse repositories, so it's only a sample of the population.
- Repositories have different levels of control over the quality of the metadata they publish. Some repositories allow anyone to publish datasets and don't intervene at all or in all cases to prevent datasets that have conflicting metadata. At the other end, some repositories have complete control. The level of control and expertise applied when publishing datasets should be taken into account if a goal of this "multiple licenses" work is to add functionality that helps depositors with any level of curation expertise easily apply Terms metadata easily that follows best practices for data sharing.

In [1]:
import csv
from functools import reduce
from google_trans_new import google_translator
import numpy as np
import pandas as pd


## Getting the data

The terms_metadata.tab file contains some basic metadata and the "Terms" metadata of all published versions of every dataset published in 49 known Dataverse repositories. Each row of the tabular file is a published version of a dataset, so there can be multiple rows (versions) per dataset. Each dataset has a unique persistentUrl, but the database ID for each dataset version, in the datasetVersionId column, is unique only within each of the 49 repositories, and many versions of different datasets in different repositories share the same datasetVersionId.

Getting the data in terms_metadata.tab:
- Download the 49 zipped files at https://doi.org/10.7910/DVN/DCDKZQ. Each zipped file contains the metadata of each published version of every dataset published in 49 known Dataverse repositories
- Using your preferred method, move all JSON files into a single folder
- Run the two scripts "get_basic_metadata.py" and "get_terms_metadata.py" at https://github.com/jggautier/dataverse_scripts/tree/master/get-dataverse-metadata/parse_metadata_fields with that folder as the input to get two CSV files, one containing the basic metadata of all datasets (publisher names, PIDs, publication dates, version numbers, etc), and one containing the Terms metadata for each version of each dataset.
- Using your preferred method, retain from the basic_metadata file only the 'publisher', 'persistentUrl', 'datasetVersionId', 'majorVersionNumber', and 'minorVersionNumber' columns.
- Using your preferred method, join both CSV files on their persistentUrl and datasetVersionId columns
- Export the results as a .tab file (or export as a CSV and convert to TAB). Because of the Dataverse software's preference for .tab files, it's easier to work with if you plan to publish this data in a Dataverse repository

In [ ]:
data = pd.read_csv('terms_metadata.tab', sep='\t', na_filter = False)

In [ ]:
# Check data
print('Number of datasets: %s' % (len(pd.unique(data['persistentUrl']))))
print('Number of dataset versions: %s' %(len(data)))
      

In [ ]:
data.head(5)


To make things simple, lets look at the metadata of only latest published version of each dataset.

In [ ]:
# Get only metadata for the latest versions of each dataset
latestversion = (data
    .iloc[data.groupby('persistentUrl')['datasetVersionId']
    .agg(pd.Series.idxmax)]
    .sort_values(by=['publisher'], inplace=False, ascending=True)
    .reset_index(drop=True, inplace=False)
)


In [ ]:
# Check data
print('Number of datasets: %s' % (len(pd.unique(latestversion['persistentUrl']))))
print('Number of dataset versions: %s' %(latestversion.shape[0]))


In [ ]:
latestversion.head(5)


To make data munging easier, let's replace any blank values with null. We also know that Dartmouth's metadata exports use the default value "Root" in their publisher field. So to identify datasets from that repository, let's replace all cases of "Root" in the "publisher" column with "Dartmouth".

In [ ]:
# Replace any blank values with NaN
latestversion = latestversion.replace(r'^\s*$', np.nan, regex=True)

# Replace publisher "Root" with Dartmouth
latestversion['publisher'] = latestversion['publisher'].replace(['Root'],'Dartmouth')


## Exploring the data

What are repositories putting in the their datasets' license fields? The Dataverse software ships with support to add a CC0 waiver, and the database stores that waiver in the "license" field (which isn't accessible by the depositor). But some repositories have forked their Dataverse installation to let depositors choose a CC BY license instead. So let's see what values in the license field we need to consider.

In [ ]:
latestversion.license.unique()

Looks like the variation is CC0, CC BY, and CCBY. The software applies "NONE" when depositors indicate that they don't want the CC0 waiver applied, at which point the Terms of Use field appears in the UI. Depositors can enter anything in that field, including any type of license, but based on community discussion the CC0 waiver and the CCBY license may be the most popular applied to datasets, so we'll narrow our exploration to only those two things.

Depositors can also enter licenses in other Terms fields, e.g. Terms of Access or Conditions, but it might be fair to assume that, most datasets with licences have that license text in their Terms of Use field, either entered automatically by the Dataverse software when the depositor chooses CC0 or CCBY, or, because of the visual prominence of the Terms of Use field, entered by the depositor when she doesn't choose CC0 or CCBY. So we'll look for datasets that have text in their Terms of Use field that indicates a CC0 waiver, a CCBY license or another creative commons license.

That is, which datasets have the CC0 waiver or CCBY licenses in their Terms of Use metadata? It should be safe and easy to assume that datasets with the words "creative commons" in their Terms of Use metadata will have some sort of Creative Commons license applied, so we'll query for that as well.

In [ ]:
data_with_licenses = (
    latestversion.query(
        'termsOfUse.str.contains("CC0")\
        or termsOfUse.str.contains("CCBY")\
        or termsOfUse.str.contains("CC BY")\
        or termsOfUse.str.contains("creative commons", case = False)'
    )
)

In [ ]:
# Check data
print('Number of datasets: %s' % (len(pd.unique(data_with_licenses['persistentUrl']))))


Of the datasets with a CC waiver or license in their ToU field, which have any text in their other Terms metadata?

In [ ]:
data_with_licenses_and_other_terms = (
    data_with_licenses.query(
        'termsOfAccess == termsOfAccess or\
        availabilityStatus == availabilityStatus or\
        citationRequirements == citationRequirements or\
        conditions == conditions or\
        confidentialityDeclaration == confidentialityDeclaration or\
        contactForAccess == contactForAccess or\
        dataaccessPlace == dataaccessPlace or\
        depositorRequirements == depositorRequirements or\
        disclaimer == disclaimer or\
        originalArchive == originalArchive or\
        restrictions == restrictions or\
        sizeOfCollection == sizeOfCollection or\
        specialPermissions == specialPermissions or\
        studyCompletion == studyCompletion'
    )
)


In [ ]:
# Check data
print('Number of datasets: %s' % (len(pd.unique(data_with_licenses_and_other_terms['persistentUrl']))))


In [ ]:
data_with_licenses_and_other_terms.head(5)

5,804 datasets fit this bill.

Now we have data we can use to answer our two questions:
- When depositors add data with CC0 waivers or CC licenses, how often are they filling the other Terms fields?
- When depositors add data with CC0 waivers or CC licenses and they fill one or more other Terms fields, are they entering things that conflict or might conflict with the waiver or CC license?

Let's try to get a sense of the variety of Terms metadata entered and what kind of grouping we can do to learn how prevelant certain behaviors are.

First let's see how many unique values exist for each field in these 5,804 datasets.

In [ ]:
termsFields = ['termsOfAccess', 'availabilityStatus', 'citationRequirements', 'conditions', 'confidentialityDeclaration', 'contactForAccess', 'dataaccessPlace', 'depositorRequirements', 'disclaimer', 'originalArchive', 'restrictions', 'sizeOfCollection', 'specialPermissions', 'studyCompletion']
for field in termsFields:
    print(field + ': ' + str(data_with_licenses_and_other_terms[field].nunique()))

So there are no values entered in dataaccessPlace. There are very few unique values entered in fields like studyCompletion and availabilityStatus. Lots of variation in termsOfAccess and citationRequirements.

Let's see how many of these datasets are published in each repository.


In [ ]:
datasetsByPublisher = data_with_licenses_and_other_terms.value_counts(subset=['publisher']).to_frame('dataset_count')
datasetsByPublisher


In the top two repositories, and 4 of the top 5, publishing datasets is less centralized, meaning control over what's published and the quailty of it is left to depositors and there's less installation-wide control over what's published. This increases the chance that the metadata, and in this case the Terms metadata, is created by depositors who don't apply a lot of knowledge about data licenses and their relationship to the Dataverse software's DDI-inspired Terms metadata fields. 

This will make generalizing more difficult, so let's narrow our exploration to a few of the top 5 repositories listed where there's no or little installation-wide control over what's published.

In [ ]:
rin_datasets = (data_with_licenses_and_other_terms.query('publisher == "RIN Dataverse"'))
print('Number of datasets: %s' %(len(rin_datasets)))


What values are entered in the license fields of the 2,278 datasets in the RIN repository?

In [ ]:
rin_datasets.license.unique()


I know that the repository hasn't forked their code to let depositors choose CCBY, so this is expected. We can assume that of these 2,278 datasets, the depositors of datasets with no license (license = NONE) must have entered CC0 or a CC license in the Terms of Use metadata field. Let's see how many datasets have in their license field "CC0" and how many have "NONE".

In [ ]:
rin_datasets.value_counts(subset=['license']).to_frame('dataset_count')


Let's explore the datasets that have CC0 in either their licence field or Terms of Use field. We'll save a dataframe with this info.

In [ ]:
rin_datasets_cc0 = (
    rin_datasets.query(
        'license == "CC0" or\
        termsOfUse.str.contains("CC0")'
    )
)
print('Number of datasets: %s' %(len(rin_datasets_cc0)))


Among the datasets we know have CC0 applied, how many unique values exist for each of the Terms fields?

In [ ]:
for field in termsFields:
    print(field + ': ' + str(rin_datasets_cc0[field].nunique()))


Looks like most of the metadata is in the termsOfAccess field. We'll list the unique values entered in that field and count how many datasets have each of those values. 

In [ ]:
rin_datasets_cc0_toa = rin_datasets_cc0.value_counts(subset=['termsOfAccess']).to_frame('dataset_count').reset_index(drop=False, inplace=False)
rin_datasets_cc0_toa.head(5)


Right, this repository is based in Indonesia, so we'll need to translate the metadata. Let's translate what's been entered into the Terms of Access fields of the 1,061 RIN datasets where CC0 has been applied.

In [ ]:
translator.translate(rin_datasets_cc0_toa['termsOfAccess'][0], lang_src='id', lang_tgt='en')


"Files downloaded from this RIN may not be redistributed in any form (electronic, electro-magnetic or printed) without the prior consent of the data distributor."

Right away, it seems like what's been entered into the Terms of Access field of at least 1061 RIN datasets conflicts with CC0, which I would assume would let the person who's downloaded the files do whatever they want with them, even without "the prior consent of the data distributor."

But this data is hosted in Indonesia, and while CC0 gives "creators a way to waive all their copyright and related rights in their works to the fullest extent allowed by law" (https://creativecommons.org/share-your-work/public-domain/cc0/), we don't know what "the fullest extent allowed by law" means in Indonesia.


Now let's look at the other RIN datasets that may be using CC licenses.


In [ ]:
rin_datasets_cc = (rin_datasets.query('license == "NONE"'))
print('Number of datasets: %s' %(len(rin_datasets_cc)))


Which CC licenses are these 48 datasets using? "NONE" is in their license fields, so there must be some text in their Terms of Use fields about a CC license:

In [ ]:
rin_datasets_cc0_tou = (rin_datasets_cc
    .value_counts(subset=['termsOfUse'])
    .to_frame('dataset_count')
    .reset_index(drop=False, inplace=False))

with pd.option_context('display.max_colwidth', -1):
    display(rin_datasets_cc0_tou)


In [ ]:
rin_datasets_cc0_tou['dataset_count'].sum()

Scanning each of the rows, we see that all 48 datasets use CCBY (with 16 datasets using the same exact text). What does the Terms metadata of those 48 datasets look like?

In [ ]:
for field in termsFields:
    print(field + ': ' + str(rin_datasets_cc[field].nunique()))


A few more fields have values, compared to RINs CC0 datasets.

Let's see what's in the Terms of Access fields first.

In [ ]:
rin_datasets_cc_toa = (rin_datasets_cc
    .value_counts(subset=['termsOfAccess'])
    .to_frame('dataset_count')
    .reset_index(drop=False, inplace=False))

with pd.option_context('display.max_colwidth', -1):
    display(rin_datasets_cc_toa)


47 of the 48 datasets have Terms of Access metadata, and most of those 47 have the same CCBY text entered in their Terms of Access fields. Scrolling down, we see 2 datasets in the fifth row with extra text in Indonedian. What does it say?


In [ ]:
translator.translate(rin_datasets_cc_toa['termsOfAccess'][4], lang_src='id', lang_tgt='en')

After the CCBY links and images, there's "Files downloaded from this RIN may not be redistributed in any form (electronic, electro-magnetic or printed) without the prior consent of the data distributor." This and the remaining text looks a lot like what was entered in the Terms Of Access fields of the CC0 datasets we looked at earlier.

And it again seems to conflict with CCBY (https://creativecommons.org/licenses/by-nc-nd/4.0/), where there are limitations on redistribution, but they don't include getting the data distributor's consent.

Let's see how many of the 47 datasets have CCBY and this text about consent. "tanpa izin dari distributor data sebelumnya" means "without prior permission of the data distributor".


In [ ]:
rin_datasets_cc_toa.query(
            'termsOfAccess.str.contains("tanpa izin dari distributor data sebelumnya")\
            and termsOfAccess.str.contains("creativecommons")'
    )


Just five. What about the other 42 datasets?

In [ ]:
# Create dataframe with datasets that have Terms of Access metadata of datasets that don't have 
df = (
    rin_datasets_cc_toa.query(
            'termsOfAccess.str.contains("tanpa izin dari distributor data sebelumnya")==False\
            and termsOfAccess.str.contains("creativecommons")==False'
    )
)

# Translate the text in the termsOfAccess field
termsOfAccessText = df['termsOfAccess'].to_list()
count = 0
for text in termsOfAccessText:
    print(str(count) + ": ", translator.translate(text, lang_src='id', lang_tgt='en'))
    count += 1
